In [1]:
spark

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 14, 6, Finished, Available)

In [2]:
## Necessary Libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import *

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 14, 7, Finished, Available)

### Loading Intermediate Data

Here we load the intermediate data which includes 2 datasets: `Comments` and `Submissions`. These datasets include all the Reddit posts between September 2021 and April 2022 (The approximate length of the College Basketball Season). No additional cleaning has already been done on this data 

In [3]:
workspace_default_storage_account = "aml6214727930"
workspace_default_container = "azureml-blobstore-6653633b-3460-4381-9199-d9e0f368353c"

workspace_wasbs_base_url = (
    f"wasbs://{workspace_default_container}@{workspace_default_storage_account}.blob.core.windows.net/"
)


StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 14, 8, Finished, Available)

In [4]:
comments_path = "/basketball_comments_sep.parquet"
submissions_path = "/basketball_submissions_sep.parquet"

comments = spark.read.parquet(f"{workspace_wasbs_base_url}{comments_path}")
submissions = spark.read.parquet(f"{workspace_wasbs_base_url}{submissions_path}")

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 14, 9, Finished, Available)

### Exploring the Data

In [7]:
submissions = submissions.cache()
comments = comments.cache()

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 14, 12, Finished, Available)

#### Data Shape and Schema

The breakdown of the 3 subreddits by number of posts is heavily imbalanced as is expected. In submissions there are just over 22,000 posts and in comments there are just over 1,575,000 comments. The majority of these posts come from the CollegeBasketball subreddit. 

In [8]:
print("Count of Submissions by Subreddit")
print(submissions.groupBy('subreddit').count().show())

print("Count of Comments by Subreddit")
print(comments.groupBy('subreddit').count().show())

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 14, 13, Finished, Available)

Count of Submissions by Subreddit
+-----------------+-----+
|        subreddit|count|
+-----------------+-----+
|         jayhawks|  632|
|CollegeBasketball|21290|
|         tarheels|  230|
+-----------------+-----+

None
Count of Comments by Subreddit
+-----------------+-------+
|        subreddit|  count|
+-----------------+-------+
|         jayhawks|   6541|
|CollegeBasketball|1567801|
|         tarheels|    828|
+-----------------+-------+

None


The important variables in the submissions dataframe are 

1. `subreddit`: The Subreddit that the post is in
2. `author` : Who authored the post
3. `selftext` : The contents of the post
4. `created_utc` : The timestamp of the post

The important variables in the comments dataframe are 

1. `subreddit`: The Subreddit that the post is in
2. `author` : Who authored the post
3. `body` : The contents of the post
4. `created_utc` : The timestamp of the post

In [12]:
print("Submissions Schema")
submissions.printSchema()

print("Comments Schema")
comments.printSchema()

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 14, 17, Finished, Available)

Submissions Schema
root
 |-- subreddit: string (nullable = true)
 |-- author: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- title: string (nullable = true)
 |-- selftext: string (nullable = true)
 |-- created_utc: timestamp (nullable = true)
 |-- num_comments: long (nullable = true)

Comments Schema
root
 |-- subreddit: string (nullable = true)
 |-- author: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- body: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- id: string (nullable = true)
 |-- created_utc: timestamp (nullable = true)



#### Missing Data

Since Reddit is a user driven platform there will likely be issues with missing text in posts or deleted posts/comments. Lets check for missing data before we proceed.

In [19]:
# Submissions
missing_values = ["", "[deleted]", "[removed]"]
missing_df = submissions.withColumn("selftext", when(col("selftext").isin(missing_values), None).otherwise(col("selftext")))
missing_data_count = missing_df.filter(col("selftext").isNull()).count()
print("Missing Text Column in Submissions =",missing_data_count)
# Comments
missing_df_c = comments.withColumn("body", when(col("body").isin(missing_values), None).otherwise(col("body")))
missing_data_count = missing_df_c.filter(col("body").isNull()).count()
print("Missing Text Column in Comments =",missing_data_count)

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 14, 24, Finished, Available)

Missing Text Column in Submissions = 12348
Missing Text Column in Comments = 85295


There are over 12,000 missing posts in `Submissions` and around 85,000 missing comments in `Comments`. We will remove these before doing further analysis

In [22]:
submissions = missing_df.na.drop(subset=["selftext"])
comments = missing_df_c.na.drop(subset = ["body"])

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 14, 27, Finished, Available)

The new counts of the subreddits with missing removed show about half of the submission data remaining, but a large portion of the comments data is intact.

In [23]:
print("Count of Submissions by Subreddit with Missing Removed")
print(submissions.groupBy('subreddit').count().show())

print("Count of Comments by Subreddit with Missing Removed")
print(comments.groupBy('subreddit').count().show())

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 14, 28, Finished, Available)

Count of Submissions by Subreddit with Missing Removed
+-----------------+-----+
|        subreddit|count|
+-----------------+-----+
|         jayhawks|  204|
|CollegeBasketball| 9567|
|         tarheels|   33|
+-----------------+-----+

None
Count of Comments by Subreddit with Missing Removed
+-----------------+-------+
|        subreddit|  count|
+-----------------+-------+
|         jayhawks|   6328|
|CollegeBasketball|1482767|
|         tarheels|    780|
+-----------------+-------+

None


#### First Snapshot of Data

Here we can see the most important columns from `submissions` and `comments`. The `selftext` and `body` columns hold the post and comments, respectively.

In [24]:
print("Submission Data")
submissions.select("subreddit", "author", "title", "selftext", "created_utc", "num_comments").show()

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 14, 29, Finished, Available)

Submission Data
+-----------------+--------------------+--------------------+--------------------+-------------------+------------+
|        subreddit|              author|               title|            selftext|        created_utc|num_comments|
+-----------------+--------------------+--------------------+--------------------+-------------------+------------+
|CollegeBasketball|             rCBBMod|Announcement: The...|Hello r/CollegeBa...|2022-03-17 14:00:50|           9|
|CollegeBasketball|       AutoModerator|[Weekly Post] Fre...|Are you new to th...|2022-04-29 15:00:11|           0|
|CollegeBasketball|       AutoModerator|[Weekly Post] Fre...|It's Free Talk Fr...|2022-04-29 15:00:11|           1|
|CollegeBasketball|              cbbBot|[Game Thread] Neb...|[Nebraska](#f/neb...|2022-02-23 00:54:05|          41|
|CollegeBasketball|              cbbBot|[Game Thread] Kan...|[Kansas State](#f...|2022-02-23 01:00:04|         626|
|CollegeBasketball|              cbbBot|[Game Thread] #2

In [28]:
print("Comments Data")
comments.select("subreddit", "author","author_flair_text", "body", "created_utc").show()

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 14, 33, Finished, Available)

Comments Data
+-----------------+-------------------+--------------------+--------------------+-------------------+
|        subreddit|             author|   author_flair_text|                body|        created_utc|
+-----------------+-------------------+--------------------+--------------------+-------------------+
|CollegeBasketball|         joeveralls|:cincinnati: :ohi...|Nothing is fallin...|2021-11-14 00:27:16|
|CollegeBasketball|       DaddyStone13|:cincinnati: :big...|lol i forgot abou...|2021-11-14 00:27:51|
|CollegeBasketball|        seahawksjoe|   :usc: USC Trojans|Also, half of the...|2021-11-14 00:28:36|
|CollegeBasketball|       CheeseAndCam|:iowastate: Iowa ...|I like how Creigh...|2021-11-14 00:30:06|
|CollegeBasketball|          [deleted]|                null|  Losing, apparently|2021-11-14 00:30:17|
|CollegeBasketball|       corranhorn57|:cincinnati: Cinc...|Oklahoma lost, th...|2021-11-14 00:30:26|
|CollegeBasketball|SwimmingInYourPants|   :usc: USC Trojans|Ah, I mi

#### Creating author flair column

`author_flair_text` is a column that contains information about the user icon of the reddit user. In most of these posts this is the team that the author supports. Here we create a new variable that indicates who the author of the comment or post supports

In [33]:
team_pattern = r":([\w\s]+):"

# Extract the team names and create the "supported_team" column
submissions = submissions.withColumn("supported_team", regexp_extract(col("author_flair_text"), team_pattern, 1))
comments = comments.withColumn("supported_team", regexp_extract(col("author_flair_text"), team_pattern, 1))

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 14, 38, Finished, Available)

If we remove the `r/tarheels` and `r/jayhawks` subreddits from our data temporarily we can get a good idea of what teams are the most supported in the user base of `r/CollegeBasektball`. 

In `submissions` the most common icons are the default `rcbb` (r/collegebasketball) and `null` (no icon). Ignoring those we see that the most supported teams are Florida State, Illinois, Purdue, and Duke. In `comments`, `null` is the most common icon, however after that the most supported teams are Purdue, North Carolina, Illinois, and Kentucky. 

This shows that some fan bases may be more active posters while others may be more involved in the comments section. 

In [39]:
print("Submissions Top Supported Teams")
submissions.filter(col("subreddit")=="CollegeBasketball").groupBy("supported_team").agg(count("*").alias("count")).orderBy(desc("count")).show()
print("Comments Top Supported Teams")
comments.filter(col("subreddit")=="CollegeBasketball").groupBy("supported_team").agg(count("*").alias("count")).orderBy(desc("count")).show()

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 14, 44, Finished, Available)

Submissions Top Supported Teams
+----------------+-----+
|  supported_team|count|
+----------------+-----+
|            rcbb| 6632|
|            null|  821|
|    floridastate|  124|
|        illinois|   89|
|          purdue|   84|
|            duke|   69|
|   northcarolina|   66|
|          kansas|   64|
|       iowastate|   64|
|        virginia|   62|
|        kentucky|   62|
|     connecticut|   51|
|          dayton|   48|
|        michigan|   43|
|mississippistate|   42|
|         indiana|   36|
|         gonzaga|   35|
|    sanjosestate|   35|
|   michiganstate|   33|
|          auburn|   32|
+----------------+-----+
only showing top 20 rows

Comments Top Supported Teams
+--------------+------+
|supported_team| count|
+--------------+------+
|          null|173474|
|        purdue| 72772|
| northcarolina| 68933|
|      illinois| 62610|
|      kentucky| 56856|
|        kansas| 55664|
|      michigan| 52395|
| michiganstate| 51169|
|       indiana| 40537|
|        auburn| 39623|
|

Using this cleaned data we will proceed to answer our business questions